### Webscrapping with Pandas pd.read_html()  + ODBC on SQL Server with pyodbc

#### Source : Wikipedia / Liste des pays par Age médian et par tranches d'âge

L'âge médian est l'âge qui divise la population en deux groupes numériquement égaux, la moitié est plus jeune et l'autre moitié est plus âgée. Il permet de mesurer le vieillissement ou le rajeunissement, et donc l'accroissement ou le déclin démographique d'un pays.

In [289]:
import numpy as np
import pandas as pd
# Beautifulsoup already installed into Anaconda environment

# Librairy PyODBC for SQL Server connection
import os
import pyodbc
# setup SQL Server connection session
server = '-PCSJN\DATAVIZ'  
database = 'Climbing_DW_BI'
connection_string= (
                    'DRIVER={SQL Server};SERVER='
                     + server
                     + ';DATABASE='
                     + database 
                     + ';Trusted_Connection=yes'                  
                    )

# Function for injecting dataframe content into SQL Server Table
def template_SQL_insert_values(table_name,dict):
    
    columns = "("
    for k in dict.keys():
        columns += str(k) + ','
    columns = columns[:-1] ### remove last comma
    columns += ")"
    
    req = f"""INSERT INTO {table_name} {columns} VALUES {tuple(dict.values())}"""
    return req

In [290]:
### 1er url
Wiki_Countries_Median_Age = "https://fr.wikipedia.org/wiki/Liste_des_pays_par_%C3%A2ge_m%C3%A9dian"
### récuperer toutes les tables html vu sur le site (url)
Dataset_Wiki_Countries_Median_Age = pd.read_html(Wiki_Countries_Median_Age)
type(Dataset_Wiki_Countries_Median_Age), len(Dataset_Wiki_Countries_Median_Age)

(list, 3)

In [291]:
Dataset_Wiki_Countries_Median_Age[0]

État ou territoire Estimation 2016[1],[2]                
            État ou territoire                  Total  Hommes  Femmes
0                        Monde                    301     294     309
1                  Afghanistan                    186     185     186
2               Afrique du Sud                    268     265     270
3                      Albanie                    325     312     338
4                      Algérie                    278     275     281
..                         ...                    ...     ...     ...
222  Wallis-et-Futuna (France)                    316     306     328
223                      Yémen                    192     191     193
224                     Zambie                    167     166     169
225                   Zimbabwe                    206     205     208
226         État ou territoire                  Total  Hommes  Femmes

[227 rows x 4 columns]

Attention : la virgule de séparation décimale a été supprimée : il y a vait un chiffre après la virgule.
Il faut diviser / 10 les valeurs scrappées.

In [292]:
Dataset_Wiki_Countries_Median_Age[0].columns

MultiIndex([(    'État ou territoire', 'État ou territoire'),
            ('Estimation 2016[1],[2]',              'Total'),
            ('Estimation 2016[1],[2]',             'Hommes'),
            ('Estimation 2016[1],[2]',             'Femmes')],
           )

In [293]:
Dataset_Wiki_Countries_Median_Age[0].columns = ['Nation','Estimation2016_Total','Estimation2016_Hommes','Estimation2016_Femmes']
Dataset_Wiki_Countries_Median_Age[0]

,Nation,Estimation2016_Total,Estimation2016_Hommes,Estimation2016_Femmes
0,Monde,301,294,309
1,Afghanistan,186,185,186
2,Afrique du Sud,268,265,270
3,Albanie,325,312,338
4,Algérie,278,275,281
...,...,...,...,...
222,Wallis-et-Futuna (France),316,306,328
223,Yémen,192,191,193
224,Zambie,167,166,169
225,Zimbabwe,206,205,208


In [294]:
Dataset_Wiki_Countries_Median_Age[0].iloc[226,1] = 1
Dataset_Wiki_Countries_Median_Age[0].iloc[226,2] = 1
Dataset_Wiki_Countries_Median_Age[0].iloc[226,3] = 1
Dataset_Wiki_Countries_Median_Age[0]


,Nation,Estimation2016_Total,Estimation2016_Hommes,Estimation2016_Femmes
0,Monde,301,294,309
1,Afghanistan,186,185,186
2,Afrique du Sud,268,265,270
3,Albanie,325,312,338
4,Algérie,278,275,281
...,...,...,...,...
222,Wallis-et-Futuna (France),316,306,328
223,Yémen,192,191,193
224,Zambie,167,166,169
225,Zimbabwe,206,205,208


In [295]:
# delete last row 
Dataset_Wiki_Countries_Median_Age[0] = Dataset_Wiki_Countries_Median_Age[0].drop(axis = 1, index = 226)
Dataset_Wiki_Countries_Median_Age[0]

,Nation,Estimation2016_Total,Estimation2016_Hommes,Estimation2016_Femmes
0,Monde,301,294,309
1,Afghanistan,186,185,186
2,Afrique du Sud,268,265,270
3,Albanie,325,312,338
4,Algérie,278,275,281
...,...,...,...,...
221,Viêt Nam,301,289,312
222,Wallis-et-Futuna (France),316,306,328
223,Yémen,192,191,193
224,Zambie,167,166,169


In [296]:
Dataset_Wiki_Countries_Median_Age[0]['Estimation2016_Total'].astype(np.uint16)
Dataset_Wiki_Countries_Median_Age[0]['Estimation2016_Hommes'].astype(np.uint16)
Dataset_Wiki_Countries_Median_Age[0]['Estimation2016_Femmes'].astype(np.uint16)

0      309
1      186
2      270
3      338
4      281
      ... 
221    312
222    328
223    193
224    169
225    208
Name: Estimation2016_Femmes, Length: 226, dtype: uint16

In [297]:
Dataset_Wiki_Countries_Median_Age[0]['Estimation2016_Total'].astype("float32")
Dataset_Wiki_Countries_Median_Age[0]['Estimation2016_Hommes'].astype("float32")
Dataset_Wiki_Countries_Median_Age[0]['Estimation2016_Femmes'].astype("float32")
Dataset_Wiki_Countries_Median_Age[0]

,Nation,Estimation2016_Total,Estimation2016_Hommes,Estimation2016_Femmes
0,Monde,301,294,309
1,Afghanistan,186,185,186
2,Afrique du Sud,268,265,270
3,Albanie,325,312,338
4,Algérie,278,275,281
...,...,...,...,...
221,Viêt Nam,301,289,312
222,Wallis-et-Futuna (France),316,306,328
223,Yémen,192,191,193
224,Zambie,167,166,169


In [298]:
# retrouver par tatonnement Invalid column name 'Côte d'Ivoire' pour script d'insertion de data ds SQL server
Dataset_Wiki_Countries_Median_Age[0].loc[53,:]

Nation                   Côte d'Ivoire
Estimation2016_Total               207
Estimation2016_Hommes              208
Estimation2016_Femmes              206
Name: 53, dtype: object

In [299]:
# Remplacement 'Côte d'Ivoire'
Dataset_Wiki_Countries_Median_Age[0].iloc[53,0] = 'Cote d Ivoire'

In [300]:
Dataset_Wiki_Countries_Median_Age[0].dtypes

Nation                   object
Estimation2016_Total     object
Estimation2016_Hommes    object
Estimation2016_Femmes    object
dtype: object

In [301]:
Wiki_Demography_perCountries_Median_Age_estimation2016_Df0 = Dataset_Wiki_Countries_Median_Age[0]

# script de création d'une table ds SQL 
create_table = """Create TABLE Wiki_Demography_perCountries_Median_Age_estimation2016_Df0
                                   ( 
                                        id INT IDENTITY PRIMARY KEY,
                                        Nation VARCHAR(100),
                                        Estimation2016_Total VARCHAR(50),
                                        Estimation2016_Hommes VARCHAR(50),
                                        Estimation2016_Femmes VARCHAR(50),                                                                                                        
                                   ) """

# launch session
cnxn = pyodbc.connect(connection_string,autocommit=True) # no cursor.commit as Autocommit already
cursor = cnxn.cursor() 

try: 
    # load script "create " 
    cursor.execute(create_table) 
    
    # population de la table ds SQL
    for i,row in Wiki_Demography_perCountries_Median_Age_estimation2016_Df0.iterrows(): 
        row_dict=row.to_dict()
        temp = template_SQL_insert_values('Wiki_Demography_perCountries_Median_Age_estimation2016_Df0',row_dict)
        cursor.execute(temp)
   
except Exception as err: 
    print('err:',err) 
try: 
    cursor.close()  # close session
except Exception as err: 
    print('err:',err)

In [302]:
Dataset_Wiki_Countries_Median_Age[1]

,État ou territoire,estimation 1955,estimation 1985,estimation 2015,projection 2030,projection 2050
0,Monde,231,233,296,330,361
1,Afghanistan,192,163,173,219,289
2,Afrique du Sud,203,196,261,296,341
3,Albanie,206,228,362,416,495
4,Algérie,186,171,275,318,370
...,...,...,...,...,...,...
111,Venezuela,173,199,274,320,379
112,Viêt Nam,236,200,304,369,420
113,Yémen,187,147,192,228,293
114,Zimbabwe,182,159,190,218,277


In [303]:
Dataset_Wiki_Countries_Median_Age[1].columns =['Nation','estimation1955','estimation1985','estimation2015','projection2030','projection2050']

In [304]:
Dataset_Wiki_Countries_Median_Age[1]

,Nation,estimation1955,estimation1985,estimation2015,projection2030,projection2050
0,Monde,231,233,296,330,361
1,Afghanistan,192,163,173,219,289
2,Afrique du Sud,203,196,261,296,341
3,Albanie,206,228,362,416,495
4,Algérie,186,171,275,318,370
...,...,...,...,...,...,...
111,Venezuela,173,199,274,320,379
112,Viêt Nam,236,200,304,369,420
113,Yémen,187,147,192,228,293
114,Zimbabwe,182,159,190,218,277


In [305]:
Dataset_Wiki_Countries_Median_Age[1] = Dataset_Wiki_Countries_Median_Age[1].drop(axis = 1, index = 115)
Dataset_Wiki_Countries_Median_Age[1]

,Nation,estimation1955,estimation1985,estimation2015,projection2030,projection2050
0,Monde,231,233,296,330,361
1,Afghanistan,192,163,173,219,289
2,Afrique du Sud,203,196,261,296,341
3,Albanie,206,228,362,416,495
4,Algérie,186,171,275,318,370
...,...,...,...,...,...,...
110,Uruguay,284,304,349,378,425
111,Venezuela,173,199,274,320,379
112,Viêt Nam,236,200,304,369,420
113,Yémen,187,147,192,228,293


In [306]:
Dataset_Wiki_Countries_Median_Age[1]['estimation1955'].astype(np.uint16)
Dataset_Wiki_Countries_Median_Age[1]['estimation1955'].astype("float32")
Dataset_Wiki_Countries_Median_Age[1]['estimation1985'].astype(np.uint16)
Dataset_Wiki_Countries_Median_Age[1]['estimation1985'].astype("float32")
Dataset_Wiki_Countries_Median_Age[1]['estimation2015'].astype(np.uint16)
Dataset_Wiki_Countries_Median_Age[1]['estimation2015'].astype("float32")
Dataset_Wiki_Countries_Median_Age[1]['estimation1955'].astype(np.uint16)
Dataset_Wiki_Countries_Median_Age[1]['estimation1955'].astype("float32")
Dataset_Wiki_Countries_Median_Age[1]['projection2030'].astype(np.uint16)
Dataset_Wiki_Countries_Median_Age[1]['projection2030'].astype("float32")
Dataset_Wiki_Countries_Median_Age[1]['projection2050'].astype(np.uint16)
Dataset_Wiki_Countries_Median_Age[1]['projection2050'].astype("float32")

0      361.0
1      289.0
2      341.0
3      495.0
4      370.0
       ...  
110    425.0
111    379.0
112    420.0
113    293.0
114    277.0
Name: projection2050, Length: 115, dtype: float32

In [307]:
Dataset_Wiki_Countries_Median_Age[1].dtypes

Nation            object
estimation1955    object
estimation1985    object
estimation2015    object
projection2030    object
projection2050    object
dtype: object

In [308]:
# retrouver par tatonnement Invalid column name 'Côte d'Ivoire' pour script d'insertion de data ds SQL server
Dataset_Wiki_Countries_Median_Age[1].iloc[29,0]

"Côte d'Ivoire"

In [309]:
# Remplacement 'Côte d'Ivoire'
Dataset_Wiki_Countries_Median_Age[1].iloc[29,0] = 'Cote d Ivoire'
Dataset_Wiki_Countries_Median_Age[1].iloc[29,0]

'Cote d Ivoire'

In [310]:
Wiki_Demography_perCountries_Median_Age_estimations_and_projections_Df1 = Dataset_Wiki_Countries_Median_Age[1]

# script de création d'une table ds SQL 
create_table = """Create TABLE Wiki_Demography_perCountries_Median_Age_estimations_and_projections_Df1
                                   ( 
                                        id INT IDENTITY PRIMARY KEY,
                                        Nation VARCHAR(100),
                                        estimation1955 FLOAT(32),
                                        estimation1985 FLOAT(32),
                                        estimation2015 FLOAT(32),
                                        projection2030 FLOAT(32),
                                        projection2050 FLOAT(32),                                                                                                
                                   ) """

# launch session
cnxn = pyodbc.connect(connection_string,autocommit=True) # no cursor.commit as Autocommit already
cursor = cnxn.cursor() 

try: 
    # load script "create " 
    cursor.execute(create_table) 
    
    # population de la table ds SQL
    for i,row in Wiki_Demography_perCountries_Median_Age_estimations_and_projections_Df1.iterrows(): 
        row_dict=row.to_dict()
        temp = template_SQL_insert_values('Wiki_Demography_perCountries_Median_Age_estimations_and_projections_Df1',row_dict)
        cursor.execute(temp)
   
except Exception as err: 
    print('err:',err) 
try: 
    cursor.close()  # close session
except Exception as err: 
    print('err:',err)

In [311]:
Dataset_Wiki_Countries_Median_Age[2]

,v · mListes de pays,v · mListes de pays.1
0,Démographie,Population 1700 1800 1907 Densité de populatio...
1,Économie,Monnaies Dette publique Dette extérieure Égali...
2,Éducation,Dépenses d'éducation Taux d'alphabétisation Cl...
3,Environnement,Performance environnementale Empreinte écologi...
4,Géographie,Continent Superficie forestière Sans cours d'e...
5,Militaire,Paix Dépenses militaires Forces militaires Ne ...
6,Politique,États membres de l'ONU Date d'acquisition de l...
7,Religion,Par religion Irréligion Bouddhisme Christianis...
8,Santé,Dépenses de santé Consommation de médicaments ...
9,Droit,Peine de mort exécutions les plus récentes Tau...


La liste des pays par tranches d'âge présente la répartition en trois classes d'âge d'une population à un instant donné : l'enfance et l'adolescence (0-14 ans), la population active (15 à 64 ans) et le troisième âge, grosso modo l'âge de la retraite dans les pays où elle est appliquée (plus de 65 ans). Les données sont issues de la Banque mondiale, en date de 2016, en pourcentage de la population totale.

In [312]:
### 2eme url
Wiki_Countries_Category_Age = "https://fr.wikipedia.org/wiki/Liste_des_pays_par_tranches_d%27%C3%A2ge"
### récuperer toutes les tables html vu sur le site (url)
Dataset_Wiki_Countries_Category_Age = pd.read_html(Wiki_Countries_Category_Age)
type(Dataset_Wiki_Countries_Category_Age), len(Dataset_Wiki_Countries_Category_Age)

(list, 2)

In [313]:
Dataset_Wiki_Countries_Category_Age[0]

Pays   Tranches d'âge                    \
                          Pays De 0 à 14 ans[1] de 15 à 64 ans[2]   
0                  Afghanistan           43,3 %            54,2 %   
1                       Égypte           33,5 %            61,2 %   
2                      Albanie           18,4 %            68,9 %   
3                      Algérie           28,9 %            65,0 %   
4                       Angola           47,6 %            50,1 %   
..                         ...              ...               ...   
172                Biélorussie           16,5 %            69,4 %   
173  République centrafricaine           38,8 %            57,3 %   
174                     Chypre           16,4 %            70,5 %   
175                       Pays   Tranches d'âge    Tranches d'âge   
176                       Pays    De 0 à 14 ans    De 15 à 64 ans   

                       
    Plus de 65 ans[3]  
0               2,5 %  
1               5,3 %  
2              12,7 %  
3               6,1 %  
4               2,3 %  
..                ...  
172            14,1 %  
173             3,9 %  
174            13,1 %  
175    Tranches d'âge  
176    Plus de 65 ans  

[177 rows x 4 columns]

In [314]:
Dataset_Wiki_Countries_Category_Age[0].columns = ['Nation','Moins_de_14_ans','de_15_à_64_ans','Plus_de_65_ans']
Dataset_Wiki_Countries_Category_Age[0]

,Nation,Moins_de_14_ans,de_15_à_64_ans,Plus_de_65_ans
0,Afghanistan,"43,3 %","54,2 %","2,5 %"
1,Égypte,"33,5 %","61,2 %","5,3 %"
2,Albanie,"18,4 %","68,9 %","12,7 %"
3,Algérie,"28,9 %","65,0 %","6,1 %"
4,Angola,"47,6 %","50,1 %","2,3 %"
...,...,...,...,...
172,Biélorussie,"16,5 %","69,4 %","14,1 %"
173,République centrafricaine,"38,8 %","57,3 %","3,9 %"
174,Chypre,"16,4 %","70,5 %","13,1 %"
175,Pays,Tranches d'âge,Tranches d'âge,Tranches d'âge


In [315]:
Dataset_Wiki_Countries_Category_Age[0].iloc[175,1] = '100 %'
Dataset_Wiki_Countries_Category_Age[0].iloc[176,1] = '100 %'
Dataset_Wiki_Countries_Category_Age[0].iloc[175,2] = '100 %'
Dataset_Wiki_Countries_Category_Age[0].iloc[176,2] = '100 %'
Dataset_Wiki_Countries_Category_Age[0].iloc[175,3] = '100 %'
Dataset_Wiki_Countries_Category_Age[0].iloc[176,3] = '100 %'
Dataset_Wiki_Countries_Category_Age[0]

,Nation,Moins_de_14_ans,de_15_à_64_ans,Plus_de_65_ans
0,Afghanistan,"43,3 %","54,2 %","2,5 %"
1,Égypte,"33,5 %","61,2 %","5,3 %"
2,Albanie,"18,4 %","68,9 %","12,7 %"
3,Algérie,"28,9 %","65,0 %","6,1 %"
4,Angola,"47,6 %","50,1 %","2,3 %"
...,...,...,...,...
172,Biélorussie,"16,5 %","69,4 %","14,1 %"
173,République centrafricaine,"38,8 %","57,3 %","3,9 %"
174,Chypre,"16,4 %","70,5 %","13,1 %"
175,Pays,100 %,100 %,100 %


In [316]:
Dataset_Wiki_Countries_Category_Age[0]['Moins_de_14_ans'] = Dataset_Wiki_Countries_Category_Age[0]['Moins_de_14_ans'].str.replace("%", "")
Dataset_Wiki_Countries_Category_Age[0]['Moins_de_14_ans']

0      43,3 
1      33,5 
2      18,4 
3      28,9 
4      47,6 
       ...  
172    16,5 
173    38,8 
174    16,4 
175     100 
176     100 
Name: Moins_de_14_ans, Length: 177, dtype: object

In [317]:
Dataset_Wiki_Countries_Category_Age[0]['de_15_à_64_ans'] = Dataset_Wiki_Countries_Category_Age[0]['de_15_à_64_ans'].str.replace("%", "")
Dataset_Wiki_Countries_Category_Age[0]['Plus_de_65_ans'] = Dataset_Wiki_Countries_Category_Age[0]['Plus_de_65_ans'].str.replace("%", "")
Dataset_Wiki_Countries_Category_Age[0]

,Nation,Moins_de_14_ans,de_15_à_64_ans,Plus_de_65_ans
0,Afghanistan,"43,3","54,2","2,5"
1,Égypte,"33,5","61,2","5,3"
2,Albanie,"18,4","68,9","12,7"
3,Algérie,"28,9","65,0","6,1"
4,Angola,"47,6","50,1","2,3"
...,...,...,...,...
172,Biélorussie,"16,5","69,4","14,1"
173,République centrafricaine,"38,8","57,3","3,9"
174,Chypre,"16,4","70,5","13,1"
175,Pays,100,100,100


In [318]:
Dataset_Wiki_Countries_Category_Age[0]['Moins_de_14_ans'] = Dataset_Wiki_Countries_Category_Age[0]['Moins_de_14_ans'].str.replace(",", "")
Dataset_Wiki_Countries_Category_Age[0]['Moins_de_14_ans']

0      433 
1      335 
2      184 
3      289 
4      476 
       ... 
172    165 
173    388 
174    164 
175    100 
176    100 
Name: Moins_de_14_ans, Length: 177, dtype: object

In [319]:
Dataset_Wiki_Countries_Category_Age[0]['de_15_à_64_ans'] = Dataset_Wiki_Countries_Category_Age[0]['de_15_à_64_ans'].str.replace(",", "")
Dataset_Wiki_Countries_Category_Age[0]['Plus_de_65_ans'] = Dataset_Wiki_Countries_Category_Age[0]['Plus_de_65_ans'].str.replace(",", "")
Dataset_Wiki_Countries_Category_Age[0]

,Nation,Moins_de_14_ans,de_15_à_64_ans,Plus_de_65_ans
0,Afghanistan,433,542,25
1,Égypte,335,612,53
2,Albanie,184,689,127
3,Algérie,289,650,61
4,Angola,476,501,23
...,...,...,...,...
172,Biélorussie,165,694,141
173,République centrafricaine,388,573,39
174,Chypre,164,705,131
175,Pays,100,100,100


Attention : la virgule de séparation décimale a été supprimée : il y a vait un chiffre après la virgule.
Il faut diviser / 10 les valeurs scrappées.

In [320]:
Dataset_Wiki_Countries_Category_Age[0]['Moins_de_14_ans'].astype(np.uint16)
Dataset_Wiki_Countries_Category_Age[0]['Moins_de_14_ans'].astype("float32")
Dataset_Wiki_Countries_Category_Age[0]['de_15_à_64_ans'].astype(np.uint16)
Dataset_Wiki_Countries_Category_Age[0]['de_15_à_64_ans'].astype("float32")
Dataset_Wiki_Countries_Category_Age[0]['Plus_de_65_ans'].astype(np.uint16)
Dataset_Wiki_Countries_Category_Age[0]['Plus_de_65_ans'].astype("float32")
Dataset_Wiki_Countries_Category_Age[0]

,Nation,Moins_de_14_ans,de_15_à_64_ans,Plus_de_65_ans
0,Afghanistan,433,542,25
1,Égypte,335,612,53
2,Albanie,184,689,127
3,Algérie,289,650,61
4,Angola,476,501,23
...,...,...,...,...
172,Biélorussie,165,694,141
173,République centrafricaine,388,573,39
174,Chypre,164,705,131
175,Pays,100,100,100


In [321]:
# retrouver par tatonnement Invalid column name 'Côte d'Ivoire' pour script d'insertion de data ds SQL server
Dataset_Wiki_Countries_Category_Age[0].loc[35,:]

Nation             Côte d'Ivoire
Moins_de_14_ans             423 
de_15_à_64_ans              546 
Plus_de_65_ans               31 
Name: 35, dtype: object

In [322]:
# Remplacement 'Côte d'Ivoire'
Dataset_Wiki_Countries_Category_Age[0].iloc[35,0] = 'Cote d Ivoire'
Dataset_Wiki_Countries_Category_Age[0].loc[35,:]

Nation             Cote d Ivoire
Moins_de_14_ans             423 
de_15_à_64_ans              546 
Plus_de_65_ans               31 
Name: 35, dtype: object

In [323]:
Dataset_Wiki_Countries_Category_Age[0].dtypes

Nation             object
Moins_de_14_ans    object
de_15_à_64_ans     object
Plus_de_65_ans     object
dtype: object

In [324]:
Wiki_Demography_perCountries_Category_Age_Df0 = Dataset_Wiki_Countries_Category_Age[0]

# script de création d'une table ds SQL 
create_table = """Create TABLE Wiki_Demography_perCountries_Category_Age_Df0
                                   ( 
                                        id INT IDENTITY PRIMARY KEY,
                                        Nation VARCHAR(200),
                                        Moins_de_14_ans VARCHAR(50),
                                        de_15_à_64_ans VARCHAR(50),
                                        Plus_de_65_ans VARCHAR(50),                                                                                                                                       
                                   ) """

# launch session
cnxn = pyodbc.connect(connection_string,autocommit=True) # no cursor.commit as Autocommit already
cursor = cnxn.cursor() 

try: 
    # load script "create " 
    cursor.execute(create_table) 
    
    # population de la table ds SQL
    for i,row in Wiki_Demography_perCountries_Category_Age_Df0.iterrows(): 
        row_dict=row.to_dict()
        temp = template_SQL_insert_values('Wiki_Demography_perCountries_Category_Age_Df0',row_dict)
        cursor.execute(temp)
   
except Exception as err: 
    print('err:',err) 
try: 
    cursor.close()  # close session
except Exception as err: 
    print('err:',err)

In [325]:
Dataset_Wiki_Countries_Category_Age[1]

,v · mListes de pays,v · mListes de pays.1
0,Démographie,Population 1700 1800 1907 Densité de populatio...
1,Économie,Monnaies Dette publique Dette extérieure Égali...
2,Éducation,Dépenses d'éducation Taux d'alphabétisation Cl...
3,Environnement,Performance environnementale Empreinte écologi...
4,Géographie,Continent Superficie forestière Sans cours d'e...
5,Militaire,Paix Dépenses militaires Forces militaires Ne ...
6,Politique,États membres de l'ONU Date d'acquisition de l...
7,Religion,Par religion Irréligion Bouddhisme Christianis...
8,Santé,Dépenses de santé Consommation de médicaments ...
9,Droit,Peine de mort exécutions les plus récentes Tau...
